In [1]:
import pandas as pd
import geopandas as gpd

#### Load Data

In [2]:
# Read in spatial files

# Check to see if the presence of network is in the zone. Find all bike intersections?
bike_network = gpd.read_file('../data/bikenetwork/Bike_Network.shp')

# Types of landuse are going to be a column
land_use = gpd.read_file('../data/landuse/land_use.shp')

# Functional Class, Speed, 
streets = gpd.read_file('../data/streets/cbb130b3-3f8e-4680-ac2c-ab1f84527d762020329-1-1mrz2ha.zxri.shp')


# StreetLight Data Files
stlight = gpd.read_file('../data/stl_data/128526_Vehicle_06092020/Shapefile/128526_Vehicle_06092020_zone_activity.shp')

In [11]:
streets.columns

Index(['OBJECTID', 'FNODE_', 'TNODE_', 'LPOLY_', 'RPOLY_', 'LENGTH', 'STCL2_',
       'STCL2_ID', 'PRE_DIR', 'ST_NAME', 'ST_TYPE', 'SEG_ID', 'ONEWAY',
       'CLASS', 'STNAME', 'CLASS1', 'SPEEDLIM', 'RECSIGN', 'SURFAWIDTH',
       'RECFACIL', 'PARKING', 'BIKENETWOR', 'LENGTH_MIL', 'PHASE2_SID',
       'SIDEWALK_I', 'COMBINED_S', 'SHAPE_LENG', 'STREET_TYP', 'SIDEWLK_WD',
       'WLK_ZN', 'CRB_RMPS', 'PD_PRIORTY', 'FEST_ST', 'BLDING_ZN', 'FURN_ZN',
       'BIKE_PARKI', 'LIGHTING', 'STREET_FUR', 'SIDEWLK_CA', 'STREET_TRE',
       'PLANTERS', 'STRMWTR_PL', 'TRASH_BINS', 'NEWSSTANDS', 'VENDORS',
       'ARCH_FEATU', 'BIKE_LANE', 'LEFT_SIDE_', 'BUFF_BIKE_', 'CNTRA_FLOW',
       'CLIMBNG_BI', 'CYCLE_TRAC', 'SHARED_USE', 'BIKE_FRIEN', 'MARKED_SHA',
       'GREEN_PAVE', 'BIKE_SIGNS', 'ON_ST_PARK', 'INST_BIKE_', 'LAY_BY_LAN',
       'LOADING_ZN', 'TRANSIT_ST', 'ALTUSES_PA', 'LANE_WIDTH', 'SPEED_REDU',
       'MEDIANS', 'CHICANES', 'BUS_LANES', 'DRIVEWAYS', 'UTILITIES',
       'CONSTRUCTI', 'STOR

#### Projection Set

In [4]:
def get_proj(gdf):
    # Retrieve coord from layer
    coords = gdf.crs['init']
    return coords
    

def set_proj(gdf, espg_coord = 'espg:4326'):
    # Set coordinate systems / meant to be used with function get_proj to set all coords to one thing.
    gdf.crs = {'init': espg_coord}

In [5]:
# Defining Coordinate systems to make sure everything is in the same one.
coord = [bike_network, land_use, streets]
main = stlight
print('Coordinate system', main.crs)
main_coords = get_proj(main)

for i in coord:
    print('Before', i.crs)
    set_proj(i, espg_coord = main_coords)
    print('After', i.crs)

Coordinate system {'init': 'epsg:4326'}
Before {'init': 'epsg:4326'}
After {'init': 'epsg:4326'}
Before {'init': 'epsg:4326'}
After {'init': 'epsg:4326'}
Before {'init': 'epsg:4326'}
After {'init': 'epsg:4326'}


#### Street Integration

In [13]:
# Spatial join spatial files to stlight for dictionary aggregation
# Example cities_with_country = geopandas.sjoin(cities, countries, how="inner", op='intersects')

streets_sj = gpd.sjoin(streets, stlight, op='intersects')
print('Streets Records:', streets.shape[0], '\nStreets Join', streets_sj.shape[0])

Streets Records: 40659 
Streets Join 210


In [9]:
# Empty Dictionary for streets
dstreets = {}

# Loop through the spatially joined dataframe
for index, row in streets_sj.iterrows():
    
    # Create first entry in the dictionary if there is not already a entry
    if row['name'] != dstreets:
        dstreets[row['name']] = {'Street_Class_1':row['CLASS1']}
    else:
        # Create dynamic naming convention for adding items into the list
        street_entry_num = len(dstreets[row['name']]) + 1
        street_entry_key = 'Street_Class_' + street_entry_num
        
        dstreets[row['name']][street_entry_key] = row['CLASS1']

        
# Dictionary to dataframe
sdf = pd.DataFrame.from_dict(dstreets, orient='index')
sdf['functional_class'] = sdf.index

# Rename columns
sdf.columns = ['functional_class', 'name']
sdf.head()

functional_class  \
50 Philadelphia Selections - 01152020 Zone 01  Collector Residential   
50 Philadelphia Selections - 01152020 Zone 02  Collector Residential   
50 Philadelphia Selections - 01152020 Zone 03  Collector Residential   
50 Philadelphia Selections - 01152020 Zone 04         Minor Arterial   
50 Philadelphia Selections - 01152020 Zone 05         Minor Arterial   

                                                                                        name  
50 Philadelphia Selections - 01152020 Zone 01  50 Philadelphia Selections - 01152020 Zone 01  
50 Philadelphia Selections - 01152020 Zone 02  50 Philadelphia Selections - 01152020 Zone 02  
50 Philadelphia Selections - 01152020 Zone 03  50 Philadelphia Selections - 01152020 Zone 03  
50 Philadelphia Selections - 01152020 Zone 04  50 Philadelphia Selections - 01152020 Zone 04  
50 Philadelphia Selections - 01152020 Zone 05  50 Philadelphia Selections - 01152020 Zone 05

In [12]:
# Empty Dictionary for streets
speed_streets = {}

# Loop through the spatially joined dataframe
for index, row in streets_sj.iterrows():
    
    # Create first entry in the dictionary if there is not already a entry
    if row['name'] != speed_streets:
        speed_streets[row['name']] = {'Street_Speed_1':row['SPEEDLIM']}
    else:
        # Create dynamic naming convention for adding items into the list
        street_entry_num = len(speed_streets[row['name']]) + 1
        street_entry_key = 'Street_Class_' + street_entry_num
        
        speed_streets[row['name']][street_entry_key] = row['SPEEDLIM']


# 
spdf = pd.DataFrame.from_dict(speed_streets, orient='index')
spdf['speed'] = sdf.index

# Rename columns
spdf.columns = ['speed', 'name']
spdf.head()

speed  \
50 Philadelphia Selections - 01152020 Zone 01    25   
50 Philadelphia Selections - 01152020 Zone 02    25   
50 Philadelphia Selections - 01152020 Zone 03    25   
50 Philadelphia Selections - 01152020 Zone 04    30   
50 Philadelphia Selections - 01152020 Zone 05    30   

                                                                                        name  
50 Philadelphia Selections - 01152020 Zone 01  50 Philadelphia Selections - 01152020 Zone 01  
50 Philadelphia Selections - 01152020 Zone 02  50 Philadelphia Selections - 01152020 Zone 02  
50 Philadelphia Selections - 01152020 Zone 03  50 Philadelphia Selections - 01152020 Zone 03  
50 Philadelphia Selections - 01152020 Zone 04  50 Philadelphia Selections - 01152020 Zone 04  
50 Philadelphia Selections - 01152020 Zone 05  50 Philadelphia Selections - 01152020 Zone 05

#### Land Use

In [14]:
"""
Key Attribute Fields
c_Dig1 - 1-digit level land use coding, numeric code.
c_Dig2 - 2-digit level land use coding, numeric code.

1.0 Residential
1.1 Residential Low Density
1.2 Residential Medium Density
1.3 Residential High Density
2.0 Commercial
2.1 Commercial Consumer
2.2 Commercial Business and Professional
2.3 Commercial Mixed Residential
3.0 Industrial
4.0 Civic/Institution
5.0 Transportation
6.0 Culture/Recreation
6.1 Culture/Amusement
6.2 Active Recreation
7.0 Park/Open Space
7.1 Park/Open Space
7.2 Cemetery
8.0 Water
9.0Vacant or Other
9.1 Vacant Land
9.2 Other/Unknown

"""
land_use_identity = {1:'Residential',
                     2:'Commercial',
                     3:'Industrial',
                     4:'Civic/Institution',
                     5:'Transportation',
                     6:'Culture/Recreation',
                     7:'Park/Open Space',
                     8:'Water',
                     9:'Vacant/Unknown'}
land_use.head()

c_dig2  objectid  c_dig1 c_dig1desc  cartodb_id c_dig2desc  c_dig3  \
0      51         3       5          5           1         51   512.0   
1      91         4       9          9           2         91   911.0   
2      81         5       8          8           3         81   811.0   
3      91         6       9          9           4         91   911.0   
4      51        13       5          5           5         51   514.0   

  c_dig3desc vacbldg    year  \
0        512    None  2015.0   
1        911    None  2015.0   
2        811    None  2015.0   
3        911    None  2015.0   
4        514    None  2015.0   

                                            geometry  
0  POLYGON ((-75.25613875472895 39.87629256208438...  
1  POLYGON ((-75.24548141380232 39.88944202591419...  
2  POLYGON ((-75.25759927317549 39.87727579735196...  
3  POLYGON ((-75.24576050470316 39.88440667345883...  
4  POLYGON ((-75.24197178770487 39.89063491213527...

In [ ]:
land_use_sj = gpd.sjoin(land_use, stlight, op='intersects')
print('Land Use Records:', land_use.shape[0], '\nLand Use Join', land_use_sj.shape[0])


land_use_identity = {1:'Residential',
                     2:'Commercial',
                     3:'Industrial',
                     4:'Civic/Institution',
                     5:'Transportation',
                     6:'Culture/Recreation',
                     7:'Park/Open Space',
                     8:'Water',
                     9:'Vacant/Unknown'}


# Empty Dictionary for streets
land_use_dict = {}

# Loop through the spatially joined dataframe
for index, row in land_use_sj.iterrows():
    
    # Create first entry in the dictionary if there is not already a entry
    if row['name'] != land_use_dict:
        
        
        land_use_sj[row['name']] =  {'Residential':0,
                                     'Commercial':0,
                                     'Industrial':0,
                                     'Civic/Institution':0,
                                     'Transportation':0,
                                     'Culture/Recreation':0,
                                     'Park/Open Space':0,
                                     'Water':0,
                                     'Vacant/Unknown':0}
        
        
    else:
        # Create dynamic naming convention for adding items into the list
        land_use_entry_num = len(land_use_dict[row['name']]) + 1
        land_use_entry_key = 'Land_Use_' + land_use_entry_num
        
        land_use_dict[row['name']][land_use_entry_key] = row['SPEEDLIM']


# 
spdf = pd.DataFrame.from_dict(speed_streets, orient='index')
spdf['speed'] = sdf.index

# Rename columns
spdf.columns = ['speed', 'name']
spdf.head()